In [ ]:
import json
import pandas as pd
import os
import numpy as np


import folium
from folium import plugins
from pprint import pprint
from folium.plugins import HeatMap

import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

import requests
from urllib.parse import urlparse


---
# <center><b>1. 파일 불러오기</b></center>
---

In [ ]:
# 파일 읽기
os.chdir('C:\민원분석참조모델\data\결과')
df_list = list(os.listdir(os.getcwd()))
index = 0
df = pd.read_csv(df_list[index], encoding='euc-kr')

In [ ]:
# 파일 개요
df.info()

In [ ]:
# 필요한 컬럼 추출
df = df[['민원인주소']]

In [ ]:
# 빈도 칼럼 만들기
df.insert(1, '빈도', df['민원인주소'])
df

In [ ]:
# 빈도
df_add = df.groupby('민원인주소', as_index=False).count()
df_add

---
# <center><b>2. 위경도 매핑</b></center>
---


In [ ]:
# 결측치 제거
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# 데이터 확인
df.info()

In [ ]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK APIKEY"})
    json_obj = result.json()
    keys = [key for key in json_obj]
    if keys[0] == 'documents':
        for document in json_obj['documents']:
            val = [document['address_name'], document['y'], document['x']]
            return val
        return ["", "", ""]
    else:
        return ["", "", ""]

In [ ]:
# 위도/경도 붙일 빈 열 만들기
df_ = df
df_['위도'] = np.nan
df_['경도'] = np.nan

In [ ]:
# 위도, 경도 구하기
for i in range(len(df_)):
    min_address = df_['민원인주소'][i]
    tmp = address_to_latlon(min_address)
    df_['위도'][i] = tmp[1]
    df_['경도'][i] = tmp[2]

In [ ]:
# 붙여넣기 매핑 (drop X)
df_.to_csv('../../result/5.민원발생주소/민원인주소_위도경도.csv', encoding='euc-kr', index=False)

---
# <center><b>3. 데이터 전처리</b></center>
---


In [ ]:
minwon = pd.read_csv('../../result/5.민원발생주소/민원인주소_위도경도_drop.csv', encoding='euc-kr')

In [ ]:
df_add = pd.merge(df_add, minwon, on='민원인주소')
df_add

In [ ]:
df_add.describe()

In [ ]:
# 파일 저장
df_add.sort_values(by='빈도', ascending=False, inplace=True)
df_add.reset_index(drop=True, inplace=True)
df_add.to_csv(f'../../result/5.민원발생주소/민원인주소_빈도_{df_list[index]}.csv', encoding='euc-kr', index=False)

---
# <center><b>4. 데이터 시각화</b></center>
---


In [ ]:
# 데이터 불러오기
df_add = pd.read_csv(
    f'../../result/5.민원발생주소/민원인주소_빈도_{df_list[index]}.csv', encoding='euc-kr')

In [ ]:
# 상위 민원발생 주소
top = df_add[:10]
top

In [ ]:
top_ = top[['민원인주소', '빈도']]
# top_.set_index(keys='민원인주소', inplace=True)
top_

In [ ]:
# 접수유형 파이차트

# 레이아웃 설정
layout = {
    'title': {
        'text': '<b>TOP10 민원발생주소 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'showlegend': True,
    'legend': {
        'title': {
            'text': '<b>주소</b>',
            'font': {
                'size': 15
            },
        },
        'borderwidth': 25,
        'bordercolor': '#fff',
        'bgcolor': '#F0F8FF',
        'orientation': 'h',
    },
    # 'autosize': True
    'width': 600,
    'height': 600,
}

plt = go.Figure()

plt.add_trace(
    go.Pie(
        labels=top_['민원인주소'],
        values=top_['빈도'],
    )
)

plt.update_traces(
    go.Pie(
        textposition='inside',
        textinfo='value+percent',
        textfont={
            'size': 14,
        },
        texttemplate="%{value}번<br>%{percent}"
    )
)

plt.update_layout(layout)
plt.show()


In [ ]:
# shp 파일
state_geo = '../지도/TL_SCCO_SIG_WGS84.json'
state_geo

In [ ]:
# json 파일 로드
with open(state_geo, encoding='utf-8') as file:
    json_file = json.load(file)

In [ ]:
# 민원 발생 주소 버블 차트
lat_mean = top['위도'].mean()
lon_mean = top['경도'].mean()

top_m = folium.Map(
    location=[lat_mean, lon_mean],
    zoom_start=11)

folium.Choropleth(
    json_file,
    fill_opacity=0.1,
    fill_color='#3186cc',
    line_opacity=1,
    line_weight=0.3,
    line_color='#3186cc'
    ).add_to(top_m)

for i in range(len(top)):
    latitude = top['위도'][i]
    longitude = top['경도'][i]
    radius = top['빈도'][i]/50
    text = top['민원인주소'][i]
    location = (latitude, longitude)
    folium.CircleMarker(
        location,
        radius=radius,
        color='#3186cc',
        fill_color='#3186cc',
        popup=text).add_to(top_m)

top_m


In [ ]:
# 민원 발생 주소 버블 차트
m = folium.Map(
    # location=[35.1721255, 129.0677072],
    location=[36, 127],
    tiles="cartodbpositron", zoom_start=7)

folium.Choropleth(
    json_file,
    fill_opacity=0.1,
    fill_color='#20B2AA',
    line_opacity=0.3,
    line_weight=0.9,
    line_color='#20B2AA').add_to(m)

for i in range(len(df_add)):
    latitude = df_add['위도'][i]
    longitude = df_add['경도'][i]
    # radius = df_add['빈도'][i]/50
    text = df_add['민원인주소'][i]
    location=(latitude, longitude)
    folium.CircleMarker(
        location,
        radius=1,
        color='#7B68EE',
        fill_color='#7B68EE',
        popup=text).add_to(m)

m.save(f'../../result/5.민원발생주소/민원인주소_버블차트_{df_list[index]}.html')


In [ ]:
# 히트맵
heat_m = folium.Map(
    location=[36, 127],
    tiles="cartodbpositron",
    zoom_start=7)

folium.Choropleth(
    json_file,
    fill_opacity=0.1,
    fill_color='#20B2AA',
    line_opacity=0.3,
    line_weight=0.9,
    line_color='#20B2AA').add_to(heat_m)

heat_df = df_add[['위도', '경도']]

heat_data = [[row['위도'],row['경도']] for index, row in heat_df.iterrows()]

HeatMap(heat_data).add_to(heat_m)

heat_m.save(f'../../result/5.민원발생주소/민원인주소_히트맵_{df_list[index]}.html')
heat_m
